In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [5]:
import matplotlib.pylab as plt
import numpy as np
import pickle, os, sys, random, logging, json, nemo
from tqdm.auto import tqdm
from Cfg import Cfg
from RecordingCorpus import RecordingCorpus
from multiprocessing import Pool

In [6]:
C = Cfg('NIST', 16000, 'amharic') 
audio_split_dir=f'{C.build_dir}/audio_split'

In [8]:
if __name__ == '__main__':
    with closing(Pool(16)) as pool:
        recordings = RecordingCorpus(C, pool)

100%|██████████| 122/122 [00:06<00:00, 19.99it/s]


In [9]:
from SplitCorpus import SplitCorpus
splits=SplitCorpus.transcript_split(C, recordings)

random.shuffle(splits.artifacts)
n_samples=len(splits.artifacts)

n_train = int(0.9*n_samples)
samples=splits.artifacts
train_samples=samples[0:n_train]
test_samples=samples[n_train:]

for (case, S) in [('train', train_samples), ('test', test_samples)]:
    manifest_fn=f'{C.build_dir}/{case}_manifest.json'
    with open(manifest_fn, 'w', encoding='utf-8') as f_manifest:
        for sample in tqdm(S):
            (_,root,(start,end))=sample.key
            audio = sample.source.value
            duration = sample.source.n_seconds
            transcript = sample.target.value
            audio_path=f'{audio_split_dir}/{root}_{start}_{end}.wav'
            metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
            json.dump(metadata, f_manifest)
            f_manifest.write('\n')